In [2]:
from __future__ import division
import pandas as pd
import gzip

def parse(path):
    g = gzip.open(path, 'rb')
    for l in g:
        yield eval(l)

def getDF(path):
    i = 0
    df = {}
    for d in parse(path):
        df[i] = d
        i += 1
    return pd.DataFrame.from_dict(df, orient='index')

df = getDF('reviews_Musical_Instruments.json.gz')
print(df.head(5))
df=df.head(50000)


       reviewerID        asin           reviewerName helpful  \
0  A1YS9MDZP93857  0006428320            John Taylor  [0, 0]   
1  A3TS466QBAWB9D  0014072149          Silver Pencil  [0, 0]   
2  A3BUDYITWUSIS7  0041291905  joyce gabriel cornett  [0, 0]   
3  A19K10Z0D2NTZK  0041913574            TexasCowboy  [0, 0]   
4  A14X336IB4JD89  0201891859                 dfjm53  [0, 1]   

                                          reviewText  overall  \
0  The portfolio is fine except for the fact that...      3.0   
1  If you are a serious violin student on a budge...      5.0   
2  This is and excellent edition and perfectly tr...      5.0   
3  Perfect for someone who is an opera fan or a w...      5.0   
4  How many Nocturnes does it contain? All of the...      1.0   

                            summary  unixReviewTime   reviewTime  
0                     Parts missing      1394496000  03 11, 2014  
1  Perform it with a friend, today!      1370476800   06 6, 2013  
2           Vivalldi's 

In [3]:
df['reviewText']=df['reviewText'].fillna("")
df['cleaned_review'] = df.reviewText.apply(lambda x: ' '.join([word for word in x.split() if not word.startswith('@')]))#remove @ from text

In [4]:
df.head()

,reviewerID,asin,reviewerName,helpful,reviewText,overall,summary,unixReviewTime,reviewTime,cleaned_review
0,A1YS9MDZP93857,0006428320,John Taylor,"[0, 0]",The portfolio is fine except for the fact that...,3.0,Parts missing,1394496000,"03 11, 2014",The portfolio is fine except for the fact that...
1,A3TS466QBAWB9D,0014072149,Silver Pencil,"[0, 0]",If you are a serious violin student on a budge...,5.0,"Perform it with a friend, today!",1370476800,"06 6, 2013",If you are a serious violin student on a budge...
2,A3BUDYITWUSIS7,0041291905,joyce gabriel cornett,"[0, 0]",This is and excellent edition and perfectly tr...,5.0,Vivalldi's Four Seasons,1381708800,"10 14, 2013",This is and excellent edition and perfectly tr...
3,A19K10Z0D2NTZK,0041913574,TexasCowboy,"[0, 0]",Perfect for someone who is an opera fan or a w...,5.0,Full score: voice and orchestra,1285200000,"09 23, 2010",Perfect for someone who is an opera fan or a w...
4,A14X336IB4JD89,0201891859,dfjm53,"[0, 1]",How many Nocturnes does it contain? All of the...,1.0,Unable to determine contents,1350432000,"10 17, 2012",How many Nocturnes does it contain? All of the...


In [5]:
def preprocess_text(sen):#remove all symbolic data 
    sentence = re.sub('(https?:\/\/)?([\da-z\.-]+)\.([a-z\.]{2,6})([\/\w \.-]*)',' ',sen) # Removing html tags
    sentence = re.sub('[^a-zA-Z]', ' ', sentence) # Remove punctuations and numbers
    sentence = re.sub(r"\s+[a-zA-Z]\s+", ' ', sentence) # Single character removal
    sentence = re.sub(r'\s+', ' ', sentence) # Removing multiple spaces
    sentence = sentence.replace("ain't", "am not").replace("aren't", "are not")
    sentence = ' '.join(text.lower() for text in sentence.split(' ')) # Lowering cases
    sw = stopwords.words('english')
    sentence = ' '.join(text for text in sentence.split() if text not in sw) #removing stopwords
    #sentence = ' '.join(text.lemmatize() for text in sentence.split()) #lemmatization
    return sentence

In [6]:
import re
import numpy as np
import pandas as pd 
import nltk
from nltk.stem import PorterStemmer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report,confusion_matrix
import seaborn as sns
from nltk.corpus import stopwords
#import plotly.graph_objs as go
#from plotly import tools
import random
from sklearn.metrics import f1_score, roc_auc_score
from sklearn.pipeline import make_pipeline
df['cleaned_Text'] = df.reviewText.apply(preprocess_text)

In [7]:
df['cleaned_Text'] = df['cleaned_Text'].apply(nltk.word_tokenize)#tokenise text

In [8]:
stemmer = PorterStemmer()
df['cleaned_Text'] = df['cleaned_Text'].apply(lambda x: [stemmer.stem(y) for y in x])#apply stemmer

In [9]:
df['cleaned_Text']

0        [portfolio, fine, except, fact, last, movement...
1        [seriou, violin, student, budget, edit, piano,...
2        [excel, edit, perfectli, true, orchestr, versi...
3        [perfect, someon, opera, fan, work, musician, ...
4        [mani, nocturn, contain, unabl, determnin, con...
                               ...                        
49995    [bought, christma, present, pick, fine, box, s...
49996                                              [great]
49997    [kinda, hard, get, bad, pick, work, fine, said...
49998    [fender, heavi, much, els, say, good, price, s...
49999    [attract, price, nice, pick, nice, feel, nice,...
Name: cleaned_Text, Length: 50000, dtype: object

In [10]:
df['cleaned_Text'] = df['cleaned_Text'].apply(lambda x: ' '.join(x))#apply count vectoriser

df['cleaned_Text']
count_vect = CountVectorizer()
counts = count_vect.fit_transform(df['cleaned_Text']);

In [11]:
transformer = TfidfTransformer().fit(counts)
counts = transformer.fit_transform(counts)

In [12]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

%matplotlib inline

import os
print(os.listdir())

import warnings
warnings.filterwarnings('ignore')
dataset = pd.read_csv("GFG1.csv")
dataset=dataset.head(50000)
Y=dataset["label"]


['.ipynb_checkpoints', 'algo - B.ipynb', 'archive.zip', 'bilstm.ipynb', 'cnn.ipynb', 'GFG1.csv', 'glove.6B.100d.txt', 'mlp.ipynb', 'reviews_Musical_Instruments.json', 'reviews_Musical_Instruments.json.gz', 'Untitled - Copy.ipynb', 'Untitled - Copy_.ipynb']


In [13]:
#X = df['cleaned_Text']


In [14]:
from sklearn.naive_bayes import MultinomialNB, GaussianNB
from sklearn.model_selection import train_test_split
from sklearn import preprocessing
 
# label_encoder object knows how to understand word labels.
label_encoder = preprocessing.LabelEncoder()
 
# Encode labels in column 'species'.
#Y= label_encoder.fit_transform(Y)

X_train, X_test, y_train, y_test = train_test_split(counts, Y, test_size=0.1,random_state=500) 

In [15]:


from sklearn.neural_network import MLPClassifier

clf = MLPClassifier(random_state=1, max_iter=300)
clf.fit(X_train, y_train)
#x1 = cv.transform(X_test)
yp=clf.predict(X_test)
from sklearn import metrics

# Model Accuracy: how often is the classifier correct?
b=float(metrics.accuracy_score(y_test, yp))
print("Accuracy:",metrics.accuracy_score(y_test, yp))
print("Precision:",metrics.precision_score(y_test, yp))

# Model Recall: what percentage of positive tuples are labelled as such?
print("Recall:",metrics.recall_score(y_test, yp))

Accuracy: 0.9188
Precision: 0.9474989665150889
Recall: 0.9679054054054054


In [18]:
from sklearn.metrics import classification_report
print(classification_report(y_test, yp))

              precision    recall  f1-score   support

           0       0.06      0.04      0.05       264
           1       0.95      0.97      0.96      4736

    accuracy                           0.92      5000
   macro avg       0.50      0.50      0.50      5000
weighted avg       0.90      0.92      0.91      5000

